In [ ]:
import pickle
import numpy as np
from src.groupedpaneldatamodels import GroupedFixedEffects, GroupedInteractiveFixedEffects
from joblib import Parallel, delayed
import cProfile

# Bonhomme Manresa

In [ ]:
def bm_run(run, G, i):
    x, y, f, g = run
    x = np.float32(x)
    y = np.atleast_3d(np.float32(y))
    f = np.float32(f)
    gfe_iterations = 10 if G < 4 else 100
    n_boot = 200 if G < 4 else 100
    m = GroupedFixedEffects(y, x, G, True, hide_progressbar=True).fit(
        max_iter=100, gfe_iterations=gfe_iterations, n_boot=n_boot
    )
    print(f"Finished run {i}")

    return m.to_dict()

In [ ]:
for n in (100, 200,):
    for t in (20, 50,):
        for G in (3, 6,):
            for k in (3,):
                print(f"\n\nRunning DGP1 with n={n}, t={t}, G={G}, k={k}")
                runs = pickle.load(open(f"development/generated_data/dgp1_n{n}_t{t}_G{G}_k{k}.pkl", "rb"))
                estimates = []
                estimates = Parallel(n_jobs=-1)(delayed(bm_run)(run, G, i) for i, run in enumerate(runs[:200]))

                with open(f"estimates/dgp1_n{n}_t{t}_G{G}_k{k}.pkl", "wb") as f:
                    pickle.dump(estimates, f)

In [ ]:
def ab_run(run, G, i):
    x, y, f, g, *_ = run
    x = np.float32(x)
    y = np.atleast_3d(np.float32(y))
    f = np.float32(f)
    gife_iterations = 10 if G < 4 else 100
    n_boot = 200 if G < 4 else 100
    m = GroupedInteractiveFixedEffects(y, x, G, True, hide_progressbar=True).fit(max_iter=100, gife_iterations=gife_iterations, n_boot=n_boot)
    print(f"Finished run {i}")

    return m.to_dict()


for n in (100, 200):
    for t in (20, 50):
        for G in (3, 6):
            for k in (3,):
                print(f"\n\nRunning DGP2 (ab) with n={n}, t={t}, G={G}, k={k}")
                runs = pickle.load(open(f"development/generated_data/dgp2_n{n}_t{t}_G{G}_k{k}.pkl", "rb"))
                estimates = []
                estimates = Parallel(n_jobs=-1)(delayed(ab_run)(run, G, i) for i, run in enumerate(runs[:200]))

                with open(f"estimates/dgp2_n{n}_t{t}_G{G}_k{k}.pkl", "wb") as f:
                    pickle.dump(estimates, f)

In [ ]:
def sj_run(run, G, i):
    x, y, f, g, *_ = run
    x = np.float32(x)
    y = np.atleast_3d(np.float32(y))
    f = np.float32(f)
    n_boot = 100 if G < 4 else 50
    kappa = 0.005 if G < 4 else 0.00005
    m = GroupedInteractiveFixedEffects(y, x, G, True, "su_ju", R=G, hide_progressbar=True).fit(
        n_boot=n_boot, only_bfgs=True, max_iter=100, kappa=kappa, tol=1e-2
    )
    print(f"Finished run {i}")

    return m.to_dict()

for n in (100, 200):
    for t in (20, 50):
        for G in (3, 6):
            for k in (3,):
                print(f"\n\nRunning DGP2 (su_ju) with n={n}, t={t}, G={G}, k={k}")
                runs = pickle.load(open(f"development/generated_data/dgp2_n{n}_t{t}_G{G}_k{k}.pkl", "rb"))
                estimates = []
                estimates = Parallel(n_jobs=-1)(delayed(sj_run)(run, G, i) for i, run in enumerate(runs[:200]))

                with open(f"estimates/dgp2_n{n}_t{t}_G{G}_k{k}_su_ju.pkl", "wb") as f:
                    pickle.dump(estimates, f)

In [ ]:
def ss_run(run, G, i):
    x, y, g = run
    x = np.float32(x)
    y = np.atleast_3d(np.float32(y))
    n_boot = 100 if G < 4 else 50
    kappa = 0.005 if G < 4 else 0.00005
    m = GroupedFixedEffects(y, x, G, True, "su_shi_phillips", hide_progressbar=True).fit(
        n_boot=n_boot, only_bfgs=True, max_iter=100, kappa=kappa, tol=1e-2
    )

    print(f"Finished run {i}")
    return m.to_dict()

for n in (100, 200):
    for t in (20, 50):
        for G in (3, 6):
            for k in (3,):
                print(f"\n\nRunning DGP3 (su_shi_phillips) with n={n}, t={t}, G={G}, k={k}")
                runs = pickle.load(open(f"development/generated_data/dgp3_n{n}_t{t}_G{G}_k{k}.pkl", "rb"))
                estimates = []
                estimates = Parallel(n_jobs=-1)(delayed(ss_run)(run, G, i) for i, run in enumerate(runs[:200]))
                with open(f"estimates/dgp3_n{n}_t{t}_G{G}_k{k}.pkl", "wb") as f:
                    pickle.dump(estimates, f)